In [106]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Preprocess Data

In [ ]:
import json
import os
import numpy as np
from tqdm.notebook import tqdm

_PATH_ = "/content/drive/MyDrive/labelled/" # 폴더 경로

file_name = []
dir_name = os.listdir(_PATH_) # 해당 경로에서 폴더 이름들 가져오기

X = np.empty(shape=(0, 974, 21, 2, 2)) # 총 n개의 단어에는 각각 974 프레임, 각 프레임에는 2개의 keypoints(왼손, 오른손), 각 keypoint에는 2개의 좌표(x, y)
Y = []

for dir in tqdm(dir_name, desc='folder', position=0):
    PATH_DIR = _PATH_ + dir # json 데이터 있는 폴더
    file_name = os.listdir(PATH_DIR) # 폴더 내 json 파일 이름들 가져오기
    for file in tqdm(file_name, desc='file', position=1, leave=False):
        file_path = PATH_DIR + '/' + file

        with open(file_path, 'rt', encoding='UTF-8') as f: # json 파일 open
            data = json.load(f)
            korean_text = data['korean_text']
            left_hand = data['left_hand']
            right_hand = data['right_hand']
            frame = data['time']

            keypoints = np.empty(shape=(0, 21, 2, 2))
            input_keypoints = np.zeros(shape=(974, 21, 2, 2)) # [0, ]
            for i in range(len(left_hand)): # 프레임마다 접근
                tmp = np.empty(shape=(0, 2, 2))
                for j in range(21):
                    pair = np.empty(shape=(2, 2))
                    left_x = 0
                    left_y = 0
                    right_x = 0
                    right_y = 0
                    left = np.empty(shape=2)
                    right = np.empty(shape=2)

                    left_x = left_hand[i][j*3]
                    left_y = left_hand[i][j*3+1]
                    left = np.array([left_x, left_y])

                    right_x = right_hand[i][j*3]
                    right_y = right_hand[i][j*3+1]
                    right = np.array([right_x, right_y])

                    pair = np.array([left, right])
                    tmp = np.vstack((tmp, [pair]))

                keypoints = np.vstack((keypoints, [tmp]))

            input_keypoints[:keypoints.shape[0]] = keypoints

        X = np.vstack((X, [input_keypoints]))
        Y.append(korean_text)

Y = np.array(Y)

print(X.shape)
np.save('/content/drive/MyDrive/' + 'X.npy', X)

## Data Load

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, GRU, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import numpy as np

_PATH_ = "/content/drive/MyDrive/npy/"

_X = np.load(_PATH_ + 'X_1.npy', allow_pickle=True)
_y = np.load(_PATH_ + 'Y_1.npy', allow_pickle=True)

## Word Embedding

In [ ]:
words = np.unique(_y)

vocab = {tkn: i+2 for i, tkn in enumerate(words)}
vocab['<unk>'] = 0 # 입력 데이터가 vocab에 없으면 <unk>. 근데 필요 없나..?
vocab['<pad>'] = 1 # 우리 모델에서 <pad> 필요 없을 것 같긴 한데..

import torch.nn as nn

embedding_layer = nn.Embedding(num_embeddings=len(vocab), embedding_dim=974, padding_idx=1)
embedding_layer = embedding_layer.weight # embedding table(텐서 형태)
embedding_np = embedding_layer.detach().numpy() # np 형태

embedded_y = []
for word in _y:
    idx = vocab[word]
    result = embedding_np[idx]
    embedded_y.append(result)
embedded_y = np.array(embedded_y)

## Data Load

In [ ]:
_X = tf.reshape(_X, (_X.shape[0], 974, -1))
_X = _X.numpy()

X_train, X_test, y_train, y_test = train_test_split(_X, embedded_y,
                     test_size=0.3,
                     random_state=1)

print(X_train.shape)
print(y_train.shape)

## Load & Train Model

In [ ]:
model = Sequential()

# input_shape(frame수, feature(21 * 2 * 2))
model.add(GRU(32, reset_after=False, return_sequences=True, input_shape=(974, 84)))
model.add(Dropout(0.2))
model.add(GRU(64, reset_after=False, return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(1, activation='softmax'))

model.summary()

model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

gru = model.fit(X_train, y_train, epochs=100, batch_size = 16, validation_data=(X_test, y_test))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(gru.history['loss'], label='train loss')
plt.plot(gru.history['val_loss'], label='validation loss')
plt.plot(gru.history['accuracy'], label='train acc')
plt.plot(gru.history['val_accuracy'], label='validation acc')

plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
# !pip install tensorflowjs
# import tensorflowjs as tfjs
# tfjs.converters.save_keras_model(model, "/content/drive/MyDrive/Colab Notebooks/WeHigher_GRU")